<a href="https://colab.research.google.com/github/gzholtkevych/URM-Programming/blob/main/URM_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1><b>Unlimited Register Machine</b></H1>

**U**nlimited **R**egister **M**achine (URM) is a realization of the approach to programming based on the automaton concept.
URM is an easier-to-understand alternative to a Turing machine.

It has (to a certain extent) the same capabilities as the Turing machine, to which URM is logically equivalent.
The URM was presented in an article published in 1963

>[John C. Shepherdson and H.E. Sturgis: *Computability of Recursive Functions*. J. ACM Vol. 10, no. 2, pp. 217–255.](https://dl.acm.org/doi/pdf/10.1145/321160.321170)

# Preparing a notebook to use

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks/Computability')

Mounted at /content/drive


In [ ]:
from typing import Dict, Any, Optional, Tuple, List
from typing_extensions import Self
from functools import reduce
from compy.nat import nat

## Memory of UMR

## Registers

URM has an enumerated set of ***registers***, each of which can store a natural number $n\in\mathbb{N}$, where $\mathbb{N}=\{0,1,2,\dotsc\} $.

Each URM program can use only a finite set of registers.

Registers are usually denoted by a capital letter $R$ with an index: $R_{0},R_{1},R_{2},\dotsc$.
The index (which is a natural number) is called the ***register index***.
The number holding register $R_{n}$ is usually denoted by $r_{n}$.

Registers are unlimited in the following two senses:

- even though a URM program can use only a finite set of registers, there is no upper limit for the number of registers used by the programs;
- there is no upper limit for numbers stored by registers.

## Memory states

The state of the memory is determined by the values ​​stored in the registers.

## Software implementation

The memory of URM is implemented as class `memory` inherited from the built-in type `list`.

The property `size` equals the length of the corresponding list.

The methods `read()` and `write()` provide correct reading from and writing to the memory.

In [ ]:
class memory(list):

    def __new__(cls) -> Self:
        return super().__new__(cls, [])

    @property
    def size(self) -> int:
        return len(self)

    def read(self, addr: int) -> nat:
        try:
            n = nat(addr)
        except ValueError:
            raise ValueError("memory.read() error! Bad argument")
        try:
            return self[n]
        except IndexError:
            return nat(0)

    def write(self, addr: int, value: Any) -> None:
        try:
            n, v = nat(addr), nat(value)
        except ValueError:
            raise ValueError("memory.write() error! Bad argument(s)")
        diff = self.size - n
        if diff <= 0:  # access to a register that is not yet allocated
            self += (1 - diff) * [nat(0)]  # allocate additional registers
        self[n] = v


## URM-programs

***Execution states***, that is, the value of the registers and the instruction counter, is changed by programs.

A ***URM program*** is a finite list of ***instructions***.

Program instructions are written in a certain order and numbered with positive natural numbers $1,2,3,\dotsc$.<br/>
**Please note!** Instruction number 0 does not exist in the program.

The instruction number for historical reasons is called the ***program line number***.

## Instruction counter

The ***instruction counter*** is an additional register that stores a natural number.
This register is denoted by $IC$ and its contents are $ic$.

<table width="800pt">
<tr><th width="100pt">Name</th>
<th width=100pt>Mnemonic code</th>
<th width="200pt">Effect</th>
<th>Description</th></tr>
<tr><td>Zero</td>
<td align="center">$\mathtt{Z}(n)$</td>
<td align="center">$R_n\gets 0$<br/>$IC\gets ic+1$</td>
<td>Replace the value in $R_{n}$ with $0$ and jump to the nextline of the<br/>program</td></tr>
<tr><td>Successor</td>
<td align="center">$\mathtt{S}(n)$</td>
<td align="center">$R_n\gets r_n+1$<br/>$IC\gets ic+1$</td>
<td>Increase the value of register $R_{n}$ by $1$ and jump to the next line of<br/>the program</td></tr>
<tr><td>Copy</td>
<td align="center">$\mathtt{C}(m,n)$</td>
<td align="center">$R_n\gets r_m$<br/>$IC\gets ic+1$</td>
<td>Replace the value in $R_{n}$ with the content of $R_{m}$ (keeping the<br/>content of $R_{m}$) and jump to the next line of the program</td></tr>
<tr><td>Jump</td>
<td align="center">$\mathtt{J}(m,n,k)$</td>
<td align="center">$IC\gets k\ \mathtt{if}\ r_m=r_n\ \mathtt{else}\ ic+1$</td>
<td>If the values ​​stored in $R_{m}$ and $R_{n}$ match, then jump to the<br/>program line with number $k$, otherwise to the next program line</td></tr>
</table>

## Software implementation

The `ins` class provides the data type implementation inhabited by the URM instructions.

- The instruction `Z(n)` is represented by a tuple `(0, n)`.
- The instruction `S(n)` is represented by a tuple `(1, n)`.
- The instruction `C(m, n)` is represented by the tuple `(2, m, n)`.
- The instruction `J(m, n, k)` is represented by the tuple `(3, m, n, k)`.

To correctly represent instructions by strings, the `__str__()` method is overloaded for this class.

The class methods `Z()`, `S()`, `C()` and `J()` provide convenient instruction creation.

In [ ]:
class ins(tuple):

    def __new__(cls, *args: Tuple[int]) -> Self:
        if not (2 <= len(args) <= 4):
            raise ValueError("ins() error! Invalid number of arguments")
        if not all(type(arg) == int for arg in args):
            raise ValueError("ins() error! Invalid type of argument(s)")
        if args[0] > 3:
            raise ValueError("ins() error! Invalid instruction code")
        if not all(arg >= 0 for arg in args[1 :]):
            raise ValueError("ins() error! Invalid value of operand(s)")
        return super().__new__(cls, args)

    def __str__(self) -> str:
        if self[0] == 0:
            return f"Z({self[1]})"
        if self[0] == 1:
            return f"S({self[1]})"
        if self[0] == 2:
            return f"C{self[1 :]}"
        # self[0] == 3
        return f"J{self[1 :]}"

    @classmethod
    def Z(cls, n: Any) -> Self:
        try:
            return ins(0, n)
        except ValueError:
            raise ValueError("Z() error! Bad operand")

    @classmethod
    def S(cls, n: Any) -> Self:
        try:
            return ins(1, n)
        except ValueError:
            raise ValueError("S() error! Bad operand")

    @classmethod
    def C(cls, m: Any, n: Any) -> Self:
        try:
            return ins(2, m, n)
        except ValueError:
            raise ValueError("C() error! Bad operand(s)")

    @classmethod
    def J(cls, m: Any, n: Any, k: Any) -> Self:
        try:
            return ins(3, m, n, k)
        except ValueError:
            raise ValueError("J() error! Bad operand(s)")


## Instructions execution

The execution of `instruction: ins` if the memory state is `memory_state` implements by calling `apply(instruction, to=memory_state)`, which provides returning the pair `(how_to_change_IC, new_memory_state)`.

The value of `how_to_change_IC` is equal to

- `None`, if after the instruction `instruction` it is necessary to execute the instruction in the next line of the program;
- the line number of the instruction that will be executed next, if it is not the next in order in the program.

The value `new_memory_state` is the state of the memory after the execution of the instruction `instruction`.

In [ ]:
def apply(i: ins, to: memory) -> Tuple[Optional[int], memory]:
    if type(to) != memory:
        raise ValueError("do() error! Invalid type of memory")
    if type(i) != ins:
        raise ValueError("do() error! Invalid type of instruction")
    if i[0] == 0:
        to.write(i[1], 0)
        return (None, to)
    if i[0] == 1:
        to.write(i[1], to.read(i[1]) + 1)
        return (None, to)
    if i[0] == 2:
        to.write(i[2], to.read(i[1]))
        return (None, to)
    # i[0] == 3
    return (i[3] if to.read(i[1]) == to.read(i[2]) else None, to)

In [ ]:
m = memory()
for n in range(10):
    m.write(n, n)
print(f"{13 * ' '}m = {m}")
next, m = apply(ins.Z(9), to=m)
print(f"next = {next}; m = {m}")
next, m = apply(ins.S(9), to=m)
print(f"next = {next}; m = {m}")
next, m = apply(ins.C(9, 0), to=m)
print(f"next = {next}; m = {m}")
next, m = apply(ins.J(1, 9, 5), to=m)
print(f"next = {next};    m = {m}")
next, m = apply(ins.J(2, 3, 10), to=m)
print(f"next = {next}; m = {m}")

             m = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
next = None; m = [0, 1, 2, 3, 4, 5, 6, 7, 8, 0]
next = None; m = [0, 1, 2, 3, 4, 5, 6, 7, 8, 1]
next = None; m = [1, 1, 2, 3, 4, 5, 6, 7, 8, 1]
next = 5;    m = [1, 1, 2, 3, 4, 5, 6, 7, 8, 1]
next = None; m = [1, 1, 2, 3, 4, 5, 6, 7, 8, 1]


# URM-programs and their execution

## Translation of a source text into an instruction list

The function `compile()` converts the text into a list of URM instructions.

In [ ]:
def compile(text: str) -> List[ins]:
    if not isinstance(text, str):
        raise ValueError("compile() error! Invalid argument type")
    # split `text` into the list of non empty lines
    lines = [line for line in [item.strip() for
                               item in text.split('\n')] if
             line]
    # the function compiles a line
    def compile_line(line: str) -> ins:
        # remove the tail of 'line' beginning with ')'
        item, sep, _ = line.partition(')')
        if not sep:  # ')' is absent in the line
            raise ValueError("compile_line() error! Bad instruction format")
        code, sep, item = item.partition('(')
        if not sep:  # '(' is absent in the line
            raise ValueError("compile_line() error! Bad instruction format")
        if code.strip() == 'Z':
            try:  # to recognize Z-instruction
                return ins.Z(int(item))
            except ValueError:
                raise ValueError("compile_line() error! Bad Z-instruction")
        elif code.strip() == 'S':
            try:  # to recognize S-instruction
                return ins.S(int(item))
            except ValueError:
                raise ValueError("compile_line() error! Bad S-instruction")
        elif code.strip() == 'C':
            op1, sep, item = item.partition(',')
            try:  # to recognize C-instruction
                return ins.C(int(op1), int(item))
            except ValueError:
                raise ValueError("compile_line() error! Bad C-instruction")
        elif code.strip() == 'J':
            op1, sep, item = item.partition(',')
            op2, sep, item = item.partition(',')
            try:  # to recognize J-instruction
                return ins.J(int(op1), int(op2), int(item))
            except:
                raise ValueError("compile_line() error! Bad J-instruction")
        raise ValueError("compile_line() error! Bad instruction format")
    # end of compile_line()
    return [compile_line(line) for line in lines]

An example of using the `compile()` function.

In [ ]:
text = """
    C(2, 0)
    Z (2)

    J(1, 2, 0)  check loop condition
        S(0)
        S(2)
    J(0, 0, 3)  repeat loop
"""

ins_lst = compile(text)
for ni, i in enumerate(ins_lst):
    print(f"{ni + 1:3d}: {i}")

  1: C(2, 0)
  2: Z(2)
  3: J(1, 2, 0)
  4: S(0)
  5: S(2)
  6: J(0, 0, 3)


## URM-programs

### Software implementation of a URM-program

In [ ]:
class program(list):

    def __new__(cls, instructions: List[ins]) -> Self:
        if not type(instructions) == list:
            raise ValueError("program() error! Invalid argument type")
        if not all(type(instruction) == ins for instruction in instructions):
            raise ValueError("program() error! Invalid type of argument member")
        return super().__new__(cls, instructions)

    def __str__(self):
        return "\n".join([f"{ni + 1: 3d}: {i}" for ni, i in enumerate(self)])

    @property
    def length(self):
        return len(self)

    @property
    def haddr(self):
        temp = 0
        for i in self:
            temp = max(temp, *i[1 : 3])
        return temp

    def get_instruction(self, lineno: int) -> Optional[ins]:
        if type(lineno) != int:
            raise ValueError("program.get_instruction() error! Bad line number")
        if lineno < 1:
            return None
        try:
            return self[lineno - 1]
        except IndexError:
            return None

In [ ]:
p = program(compile(text))
print(f"Program text:\n{p}")
print(f"Program length = {p.length}")
print(f"Program highest address = {p.haddr}")

Program text:
  1: C(2, 0)
  2: Z(2)
  3: J(1, 2, 0)
  4: S(0)
  5: S(2)
  6: J(0, 0, 3)
Program length = 6
Program highest address = 2


### Execution of a URM-program

In [ ]:
def run(prgm: program, *data: Tuple[Any]) -> nat:
    if type(prgm) != program:
        raise ValueError("run() error! Bad program")
    try:
        data = (nat(x) for x in data)
    except:
        raise ValueError("run() error! Invalid data")
    m = memory()
    for addr, value in enumerate(data):
        m.write(addr + 1, value)
    ic = 1
    while True:
        i = prgm.get_instruction(ic)
        if i is None:
            return m.read(0)
        q, m = apply(i, to=m)
        ic = ic + 1 if q is None else q

In [ ]:
report = [(x, y, run(p, x, y)) for x in range(10) for y in range(10)]
report_items = [" ".join(
    (f"{x}", "+", f"{y}", "=", f"{z:2d}")) for (x, y, z) in report]
report_lines = [" : ".join(report_items[ic + c * 20] for c in range(5)) for
                ic in range(20)]
for line in report_lines:
    print(line)

0 + 0 =  0 : 2 + 0 =  2 : 4 + 0 =  4 : 6 + 0 =  6 : 8 + 0 =  8
0 + 1 =  1 : 2 + 1 =  3 : 4 + 1 =  5 : 6 + 1 =  7 : 8 + 1 =  9
0 + 2 =  2 : 2 + 2 =  4 : 4 + 2 =  6 : 6 + 2 =  8 : 8 + 2 = 10
0 + 3 =  3 : 2 + 3 =  5 : 4 + 3 =  7 : 6 + 3 =  9 : 8 + 3 = 11
0 + 4 =  4 : 2 + 4 =  6 : 4 + 4 =  8 : 6 + 4 = 10 : 8 + 4 = 12
0 + 5 =  5 : 2 + 5 =  7 : 4 + 5 =  9 : 6 + 5 = 11 : 8 + 5 = 13
0 + 6 =  6 : 2 + 6 =  8 : 4 + 6 = 10 : 6 + 6 = 12 : 8 + 6 = 14
0 + 7 =  7 : 2 + 7 =  9 : 4 + 7 = 11 : 6 + 7 = 13 : 8 + 7 = 15
0 + 8 =  8 : 2 + 8 = 10 : 4 + 8 = 12 : 6 + 8 = 14 : 8 + 8 = 16
0 + 9 =  9 : 2 + 9 = 11 : 4 + 9 = 13 : 6 + 9 = 15 : 8 + 9 = 17
1 + 0 =  1 : 3 + 0 =  3 : 5 + 0 =  5 : 7 + 0 =  7 : 9 + 0 =  9
1 + 1 =  2 : 3 + 1 =  4 : 5 + 1 =  6 : 7 + 1 =  8 : 9 + 1 = 10
1 + 2 =  3 : 3 + 2 =  5 : 5 + 2 =  7 : 7 + 2 =  9 : 9 + 2 = 11
1 + 3 =  4 : 3 + 3 =  6 : 5 + 3 =  8 : 7 + 3 = 10 : 9 + 3 = 12
1 + 4 =  5 : 3 + 4 =  7 : 5 + 4 =  9 : 7 + 4 = 11 : 9 + 4 = 13
1 + 5 =  6 : 3 + 5 =  8 : 5 + 5 = 10 : 7 + 5 = 12 : 9 +